In [1]:
import pandas as pd
import numpy as np
import BostonHousing

In [2]:
## Our test train split implementation test
Data = np.array([[1,2],[3,4],[5,6],[6,7],[8.9],[10,11],[12,13],[14,15]])
train_test, test_set = BostonHousing.data_split(Data,0.33)
print(train_test)
print(test_set)

[[14 15]
 [10 11]
 [ 5  6]
 [ 1  2]
 [12 13]]
[array([8.9]) array([3, 4]) array([6, 7])]


/home/apostolos/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/home/apostolos/Desktop/Supervised Learning/Coursework/Supervised-Lerning---Coursework 1/BostonHousing.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(train_data), np.array(test_data)


In [3]:
dataset_file = 'BostonDataset.csv'

In [4]:
## load the dataset into a pandas dataframe
data_frame = pd.read_csv(dataset_file)  

In [5]:
data_frame.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


In [6]:
print(data_frame.columns)

Index(['CRIM', ' ZN ', 'INDUS ', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'LSTAT', 'MEDV'],
      dtype='object')


In [7]:
# convert dataframe to np array
data_array = data_frame.to_numpy()
print(data_array.shape)

(506, 13)


In [8]:
# number of runs for each experiment (defined on the assignement paper)
N = 20
features_number = 13
test_split = round(((len(data_array)/3)/len(data_array)*100)/100,2)
print("Using " + str(test_split*100) + "% of the dataset dor testing and the rest " + str(round((1 - test_split),2)*100) + "% for the training")

Using 33.0% of the dataset dor testing and the rest 67.0% for the training


In [9]:
BostonHousing.fit_with_constant_function(features_number,data_array,test_split,N)

Average MSE - std for train and test set are: 84.83199359560042 - 4.735650363472384 and 83.78677439055113 - 9.628765397970664 respectively


In [10]:
BostonHousing.fit_with_one_feature(features_number,data_array,test_split,data_frame.columns,N)

Linear Regression with single Features - Results
Linear regression using attribute: CRIM MSE train - std: 72.38468618432714 - 3.5237629041417815 MSE test - std:71.4268221642585 - 7.552055853696609
Linear regression using attribute:  ZN  MSE train - std: 74.66053646741219 - 3.790885009552597 MSE test - std:71.4232939103193 - 7.595277019809676
Linear regression using attribute: INDUS  MSE train - std: 65.13442075320768 - 3.6620859655643843 MSE test - std:64.12061987721168 - 7.439693511342923
Linear regression using attribute: CHAS MSE train - std: 82.6770822234885 - 4.843722691722229 MSE test - std:80.91631148288302 - 9.717368907451874
Linear regression using attribute: NOX MSE train - std: 69.53704622241328 - 4.104869327567475 MSE test - std:68.31002053607311 - 8.239049574763577
Linear regression using attribute: RM MSE train - std: 43.13343501896928 - 4.126267667773965 MSE test - std:44.991728507105314 - 8.795496829619394
Linear regression using attribute: AGE MSE train - std: 72.92345

In [11]:
BostonHousing.fit_with_all_features(features_number,data_array,test_split,N)

Linear regression using all the features with MSE - std train: 21.87002203494614-2.506606448243449 MSE - std test: 25.048359458139075-5.552157845578467
